In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Setting up a blended benchmark

Demonstration of how to load a blended benchmark. 
We also show how floating weights with a periodic reset.

Attributes
----------
Reference portfolios
Securitised portfolios
Weights
Floating weights
"""

toggle_code("Toggle Docstring")

# Setting up a Blended Benchmark with Floating Weights
---

This notebook shows how to setup a blended benchmarks in LUSID with floating weights, using reference portfoliios. In the example, the blended benchmark is a reference portfolio which holds two other constituent securitized portfolios. We show how the floating weights of the constituents change over time. We also show how you can configure the floating weights to reset/re-balance on a set date.

Section (1) and (2) will walk you through the process of setting up the variables, instruments and environment required for creating reference portfolios in LUSID. If instrument masters already exists in LUSID then skip to section (3) where the reference portfolios are created.

## Table of Contents

- 1. [Setup](#1.-Setup)
    * [1.1 Imports and LUSID Dependencies](#1.1-Imports-and-LUSID-Dependencies)
    * [1.2 Define Global Variables](#1.2-Define-Global-Variables)
        * [1.2.1 Specify Project Scope and Start Date](#1.2.1-Specify-Project-Scope-and-Start-Date)
        * [1.2.2 Define LUSID API's](#1.2.2-Define-LUSID-API's)

- 2. [Load Instrument Master](#2.-Load-Instrument-Master)
    * [2.1 Load the Equity Instruments](#2.1-Load-the-Equity-Instruments)
        * [2.1.1 Read Equity CSV File](#2.1.1-Read–Equity-CSV-File)
        * [2.1.2 Load Equity Data to LUSID](#2.1.2-Load-Equity-Data-to-LUSID)
    * [2.2 Load the Fixed Income instruments](#2.2-Load-the-Fixed-Income-instruments)
        * [2.2.1 Read Fixed Income CSV File](#2.2.1-Read-Fixed-Income-CSV-File)
        * [2.2.2 Load Fixed Income Data to LUSID](#2.2.2–Load-Fixed-Income-Data-to-LUSID)

- 3. [Create reference portfolios](#3.-Create-reference-portfolios)
    * [3.1 Define Names for Reference Portfolios](#3.1-Define-Names-for-Reference-Portfolios)
    * [3.2 Create Reference Portfolios](#3.2-Create-Reference-Portfolios)

- 4. [Securitise the reference portfolios](#4.-Securitise-the-reference-portfolios)

- 5. [Load constituents](#5.-Load-constituents)
    * [5.1 Load constituents for FI and EQ reference portfolios](#5.1-Load-constituents-for-FI-and-EQ-reference-portfolios)
    * [5.2 Load constituents for blended benchmark](#5.2-Load-constituents-for-blended-benchmark)

- 6. [Upsert quotes](#6.-Upsert-quotes)

- 7. [Get blended index floating weight over timeline](#7.-Get-blended-index-floating-weight-over-timeline)
    * [7.1 Get weights for 1 Jan](#7.1-Get-weights-for-1-Jan)
    * [7.2 Get weights for 31 Jan](#7.2-Get-weights-for-31-Jan)
    * [7.3 Get weights for 31 March](#7.3-Get-weights-for-31-March)
    * [7.4 Weights are reset on 1 April](#7.4-Weights-are-reset-on-1-April)

# 1. Setup
---

This notebook begins be setting up the requirements to run LUSID.

## 1.1 Imports and LUSID Dependencies

In [2]:
# To begin, we initialize global variables, objects and datasets. 
# We also load the variouse packages required for constructing our examples:

# Import general purpose packages
import os
import json
import pandas as pd
import numpy as np
import datetime
import pytz
import warnings

warnings.filterwarnings("ignore", module="matplotlib*")

import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from pandas import json_normalize
from flatten_json import flatten

# Import lusid specific packages
import lusid
import lusid.models as models
from lusidtools.cocoon.cocoon import load_from_data_frame

from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken

# Set display configuration
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.5f" % x)
pd.set_option("display.max_rows", 3500)
pd.set_option("max_colwidth", 20)

# Use line magic function to enable matplotlib to work interactively with iPython
%matplotlib inline

# Set style to fivethirtyeight to create clean and clear looking graphs
plt.style.use("fivethirtyeight")

# Define a dictionary containing default plotting configurations
params = {
    "legend.fontsize": "small",
    "figure.figsize": (12, 4.5),
    "axes.labelsize": "small",
    "axes.titlesize": "medium",
    "xtick.labelsize": "small",
    "ytick.labelsize": "small",
}

plt.rcParams.update(params)

os.environ["FBN_SECRETS_PATH"] = "/Users/tobiadeniyi/Work/secrets/secrets.json"
# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

# %load_ext lab_black
# %load_ext nb_black

,api_version,build_version,excel_version,links
0,v0,0.6.9316.0,0.5.2870,{'relation': 'Re...


## 1.2 Define Global Variables

### 1.2.1 Specify Notebook Scope and Start Date

The `scope` and  `start_date` will be used gloabally accross the Notebook.

In [3]:
# Define a scope to hold data

scope = "ukIBOR"
start_date = "2021-01-01"

### 1.2.2 Define LUSID API's

In [4]:
# Define the APIs we use

reference_portfolios_api = api_factory.build(lusid.api.ReferencePortfolioApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)

# 2. Load Instrument Master
---

Our Instrument Master data will be loaded from two seperate data sources, each corresponding to a reference portfolio (defined by the weighting of each asset). Each data source containing a different class of instrument:

1. The first containing UK equities, loaded form `"data/benchmark/uk-stocks.csv"`
2. The second containing UK fixed income, loaded from `"data/benchmark/uk-bonds.csv"`

## 2.1 Load the Equity Instruments

We begin by loading our equity instruments and mapping the accompanying identifiers and required fields to LUSID.

Our portfolio consists of a collection of various UK equities, weighted according to their proportion of the underlying portfolio. Each entry cointains a `"Name"` attribute and a conresponding `"Sector"` property. We also have three identifiers for each equity, a `"Ticker"`, `"ISIN"` and `"SEDOL"`. Each of this fields can be mapped to a properties in LUSID.

### 2.1.1 Read Equity CSV File

In [5]:
uk_stocks = pd.read_csv("data/benchmark/uk-stocks.csv")
uk_stocks.head(3)

,Ticker,Name,Sector,ISIN,SEDOL,Weighting
0,III LN,3i,Financial Services,GB00B1YW4409,B1YW440,0.04000
1,BKG LN,Berkeley Group H...,Household Goods ...,GB00B02L3W35,B02L3W3,0.04000
2,BATS LN,British American...,Tobacco,GB0002875804,287580,0.04000


### 2.1.2 Load Equity Data to LUSID

In [6]:
# Load the instruments into LUSID

instrument_identifier_mapping = {
    "ClientInternal": "Ticker",
    "Isin": "ISIN",
    "Sedol": "SEDOL",
}

instrument_mapping_required = {"name": "Name"}

instrument_mapping_optional = {}

responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=uk_stocks,
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
    property_columns=["Sector"],
)

## 2.2 Load the Fixed Income instruments

In a similar fashion, we continue by loading our fixed income data to LUSID. Our fixed income data comes with only a single identifiying field - `"ISIN"` - as well as a `"Name"` and `"Sector"` property.

### 2.2.1 Read Fixed Income CSV File

In [7]:
uk_bonds = pd.read_csv("data/benchmark/uk-bonds.csv")
uk_bonds.head(3)

,ISIN,Name,Sector,Weighting
0,GB00BNNGP668,UKT 0 ⅜ 10/22/26,Government Bond,0.05000
1,GB00BNNGP775,UKT 0 ⅞ 01/31/46,Government Bond,0.10000
2,GB00BMBL1F74,UKT 0 ⅝ 10/22/50,Government Bond,0.15000


### 2.2.2 Load Fixed Income Data to LUSID

In [8]:
# Load the instruments into LUSID

instrument_identifier_mapping = {
    "ClientInternal": "ISIN",
    "Isin": "ISIN",
}

instrument_mapping_required = {"name": "Name"}

instrument_mapping_optional = {}

responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=uk_bonds,
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
    property_columns=["Sector"],
)

# 3. Create reference portfolios
---

Next, we construct our reference portfolios. These include:

1. Our UK Equity portfolio, consiting of various weights of UK equities
2. Our UK Fixed Income porfolio, consisting of various weights of UK Bonds
3. And our Blended Portfolio, which constitutes some portion of our Equity and Fixed Income portfolios

## 3.1 Define Names for Reference Portfolios

In [9]:
uk_bond_index = "uKBondIndex"
uk_equity_index = "uKEquityIndex"
uk_blended_index = "ukBlendedIndex"

# Define our list of Reference portfolios
reference_portfolios = [uk_bond_index, uk_equity_index, uk_blended_index]

## 3.2 Create Reference Portfolios

For each reference portfolio create an instance of the portfolio in LUSID. Specify the `scope` and time `created` to be the values specified gloabally in our Notebook. A reference portfolio in LUSID allows the storage of a portfolio which contains a set of constituent instruments and weights which can change over time.

For additional information on Reference Portfolios, please see the following [KB article](https://support.lusid.com/knowledgebase/article/KA-01852/en-us).

> NOTE: The time when a portfolio is `created` most occure before or at the point when that any of it's constituents are upserted

In [10]:
# The `create_reference_portfolio()` method implements LUSID API `CreateReferencePortfolio` method. 
# More information on this API method can be found in the following [LUSID API Docs](https://www.lusid.com/docs/api/#tag/Reference-Portfolio)

for portfolio in reference_portfolios:

    try:

        response = reference_portfolios_api.create_reference_portfolio(
            scope=scope,
            create_reference_portfolio_request=models.CreateReferencePortfolioRequest(
                display_name=portfolio, code=portfolio, created="2010-01-01"
            ),
        )

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

Could not create a portfolio with id 'uKBondIndex' because it already exists in scope 'ukIBOR'.
Could not create a portfolio with id 'uKEquityIndex' because it already exists in scope 'ukIBOR'.
Could not create a portfolio with id 'ukBlendedIndex' because it already exists in scope 'ukIBOR'.


# 4. Securitise the reference portfolios
---

We now securitise each of the reference portfolios, allowing us to generate a blended benchmark containing multiple reference portfolios reperesented in LUSID. The reference portfolio can be configured to automatically float the weights of the underlying indices (the securitised reference portfolio constituents) so that they move in line with the movements of the index.

For additional information on the Securitisation of Reference Portfolios, please see the following [KB article](https://support.lusid.com/knowledgebase/article/KA-01852/en-us)

In [11]:
for portfolio in reference_portfolios:

    response = instruments_api.upsert_instruments(
        request_body={
            "upsert_instrument": models.InstrumentDefinition(
                name=portfolio,
                identifiers={
                    "ClientInternal": models.InstrumentIdValue(value=portfolio)
                },
                look_through_portfolio_id=models.ResourceId(
                    scope=scope, code=portfolio
                ),
            )
        }
    )

# 5. Load constituents
---

## 5.1 Load constituents for FI and EQ reference portfolios

We now create Index constituents for the time period in consideration for each of the reference portfolios. 

These constituents should not overlap (i.e. be on the same instrument) with any existing constituents.

In [12]:
for portfolio, df, instrument_id in [
    (uk_bond_index, uk_bonds, "ISIN"),
    (uk_equity_index, uk_stocks, "Ticker"),
]:

    # Initialise a list to hold our constituents
    constituents = []

    # Iterate over instrument unvierse to add each constituent to our list
    for _, row in df.iterrows():
        constituents.append(
            models.ReferencePortfolioConstituentRequest(
                instrument_identifiers={
                    "Instrument/default/ClientInternal": row[instrument_id]
                },
                weight=row["Weighting"],
                currency="GBP",
            )
        )

    # Create our request to add our constituents
    constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
        effective_from=start_date,
        weight_type="Periodical",
        period_type="Quarterly",
        period_count=1,
        constituents=constituents,
    )

    # Call LUSID to upsert our constituents into our reference portfolio
    response = reference_portfolios_api.upsert_reference_portfolio_constituents(
        scope=scope,
        code=portfolio,
        upsert_reference_portfolio_constituents_request=constituents_request,
    )

    print(f"Constituents Upserted for {portfolio}")

Constituents Upserted for uKBondIndex
Constituents Upserted for uKEquityIndex


## 5.2 Load constituents for blended benchmark

Now, we create a set of constituents for our blended benchmark reference portfolio. The blended portfolio is made up by 60% of the UK bond portfolio and 40% of the UK equity portfolio. So we wish to re distribute our portfolio holding to meet the specified weighing values. We also model the constituent rule to be applied periodically on a quaterly bases.

In [13]:
weightings = [(uk_bond_index, 0.6), (uk_equity_index, 0.4)]

In [14]:
# We specify quarterly rebalancing of the blended benchmark indexby specifying `weight_type` and `period_type` as shown. 
# The period count specifies how many `period_type` duration must pass before portfolio is rebalanced. 
# Additional details on reference portfolio constituents can be found in the following [KB Article](https://support.lusid.com/knowledgebase/article/KA-01852/en-us)

# Initialise a list to hold our constituents
constituents = []

# Iterate over instrument unvierse to add each constituent to our list
for port, weighting in weightings:
    constituents.append(
        models.ReferencePortfolioConstituentRequest(
            instrument_identifiers={"Instrument/default/ClientInternal": port},
            weight=weighting,
            currency="GBP",
        )
    )

# Create our request to add our constituents
constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
    effective_from=start_date,
    weight_type="Periodical",
    period_type="Quarterly",
    period_count=1,
    constituents=constituents,
)

# Call LUSID to upsert our constituents into our reference portfolio
response = reference_portfolios_api.upsert_reference_portfolio_constituents(
    scope=scope,
    code=uk_blended_index,
    upsert_reference_portfolio_constituents_request=constituents_request,
)

print(f"Constituents Upserted for {weightings[0][0]} and {weightings[1][0]}")

Constituents Upserted for uKBondIndex and uKEquityIndex


# 6. Upsert quotes
---

We now load in the relevant market prices from 1st of Jan to 1st of Apr 2021.

In [15]:
luids = {}

for instrument_id in [uk_bond_index, uk_equity_index]:

    get_luid = instruments_api.get_instrument(
        identifier_type="ClientInternal", identifier=instrument_id
    ).lusid_instrument_id

    luids[instrument_id] = get_luid

In [16]:
for date, instrument, price in [
    ("2021-01-01", uk_bond_index, 100),
    ("2021-01-31", uk_bond_index, 130),
    ("2021-03-31", uk_bond_index, 129),
    ("2021-04-01", uk_bond_index, 136),
    ("2021-01-01", uk_equity_index, 100),
    ("2021-01-31", uk_equity_index, 80),
    ("2021-03-31", uk_equity_index, 90),
    ("2021-04-01", uk_equity_index, 86),
]:

    # Create quote requests
    instrument_quotes = {
        "upsert_request_1": models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=luids[instrument],
                    instrument_id_type="LusidInstrumentId",
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=date,
            ),
            metric_value=models.MetricValue(value=price, unit="GBP"),
        )
    }

    # Upsert the quotes into LUSID
    response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

# 7. Get blended index floating weight over timeline
---

In this section, we track the benchmark weights across a Q1 timeline, with the floating weights resetting on 1 April.

## 7.1 Get weights for 1 Jan

* On the 1 Jan, which is Day 1 for the portfolio, the fixed weight is equal to the floating weight

In [17]:
get_constituents = reference_portfolios_api.get_reference_portfolio_constituents(
    scope=scope, code=uk_blended_index, effective_at="2021-01-01"
)

pd.DataFrame([flatten(item.to_dict()) for item in get_constituents.constituents])

,instrument_identifiers_Instrument/default/ClientInternal,instrument_uid,currency,properties,weight,floating_weight,instrument_scope
0,uKBondIndex,LUID_00003DEW,GBP,{},0.60000,0.60000,default
1,uKEquityIndex,LUID_00003DEX,GBP,{},0.40000,0.40000,default


## 7.2 Get weights for 31 Jan

* The floating weights have moved as the underlying indicies have moved
    * The bond portfolio has performed well, meaning the float weight has jumpted from 60% to ~ 70%
    * The equity portfolio has not performed as well, meaning its floating weight has dropped to ~30%

In [18]:
get_constituents = reference_portfolios_api.get_reference_portfolio_constituents(
    scope=scope, code=uk_blended_index, effective_at="2021-01-31"
)

pd.DataFrame([flatten(item.to_dict()) for item in get_constituents.constituents])

,instrument_identifiers_Instrument/default/ClientInternal,instrument_uid,currency,properties,weight,floating_weight,instrument_scope
0,uKBondIndex,LUID_00003DEW,GBP,{},0.60000,0.70909,default
1,uKEquityIndex,LUID_00003DEX,GBP,{},0.40000,0.29091,default


## 7.3 Get weights for 31 March

* The floating weights have continued to move

In [19]:
get_constituents = reference_portfolios_api.get_reference_portfolio_constituents(
    scope=scope, code=uk_blended_index, effective_at="2021-03-31"
)

pd.DataFrame([flatten(item.to_dict()) for item in get_constituents.constituents])

,instrument_identifiers_Instrument/default/ClientInternal,instrument_uid,currency,properties,weight,floating_weight,instrument_scope
0,uKBondIndex,LUID_00003DEW,GBP,{},0.60000,0.68254,default
1,uKEquityIndex,LUID_00003DEX,GBP,{},0.40000,0.31746,default


## 7.4 Weights are reset on 1 April

On the 1 April, the weights are reset, and the weight and floating rate are equal.

In [20]:
get_constituents = reference_portfolios_api.get_reference_portfolio_constituents(
    scope=scope, code=uk_blended_index, effective_at="2021-04-01"
)

pd.DataFrame([flatten(item.to_dict()) for item in get_constituents.constituents])

,instrument_identifiers_Instrument/default/ClientInternal,instrument_uid,currency,properties,weight,floating_weight,instrument_scope
0,uKBondIndex,LUID_00003DEW,GBP,{},0.60000,0.60000,default
1,uKEquityIndex,LUID_00003DEX,GBP,{},0.40000,0.40000,default
